# GrobidClient Interactive Playground

This notebook demonstrates how to use `grobid-client-python` to process a PDF and extract its fulltext XML output.

In [2]:
# Import Required Libraries
import sys
from grobid_client.grobid_client import GrobidClient

In [7]:
# Initialize GrobidClient
client = GrobidClient(grobid_server="http://localhost:8070", config_path="/home/jakub/Projects/paper_testing/config.json")  # use Docker container URL

GROBID server is up and running


In [8]:
# Process PDF to Extract Fulltext
pdf_path = "/home/jakub/Projects/paper_testing/documents_to_test"  # Change this to your PDF file
output_dir = "output"  # Output directory for results

result = client.process("processFulltextDocument", pdf_path, output=output_dir, consolidate_citations=True, tei_coordinates=True, force=True, include_raw_citations=True, include_raw_affiliations=True)

Processing of /home/jakub/Projects/paper_testing/documents_to_test/pdfs/1-s2.0-S0747563221004325-main.pdf failed with error 500 , [NO_BLOCKS] PDF parsing resulted in empty content
Processing of /home/jakub/Projects/paper_testing/documents_to_test/pdfs/2025.06.13.25329541v2.full.pdf failed with error 408 , None
Processing of /home/jakub/Projects/paper_testing/documents_to_test/pdfs/jmir-2020-10-e17757.pdf failed with error 408 , None
Processing of /home/jakub/Projects/paper_testing/documents_to_test/pdfs/kelders-et-al-2024-what-does-it-mean-to-be-engaged-with-digital-health-interventions-a-qualitative-study-into-the.pdf failed with error 408 , None


**Note:** grobid parsing failed for 1-s2.0-S0747563221004325-main.pdf, marking it as a damaged pdf

In [27]:
# Display Output Path
print(f"GROBID output saved to: {result}")

GROBID output saved to: None
